# Import data

In [126]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit



url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
data=pd.read_csv(url)
data_format=data.groupby('Country/Region', as_index=False).sum().drop(['Lat','Long'],axis=1)
#columns={i:data_format[i][0] for i in range(np.shape(data_format)[1])}
#data_format=data_format.rename(columns=columns)

world_data=pd.read_csv('world_data.csv',sep=';',thousands=',', decimal='.')


# cummulated number of deaths

## Raw data plot curves

In [124]:
fig = go.Figure()
for row in data_format.iterrows():
    to_keep=row[1][1:].values>1
    if any(row[1][1:].values>1000):
        fig.add_trace(go.Scatter(x=np.array(range(-1,to_keep.sum())), y=np.append([0],row[1][1:].values[to_keep]),
                        mode='lines',
                        name=row[1][0]))
    #else :
        #print(row[1][0])
fig.update_layout(title='Total number of deaths per country',
                   xaxis_title='Days after the first reported death',
                   yaxis_title='Number of deaths')
fig.show();

## Smoothed curves

In [189]:
fig = go.Figure()
for row in data_format.iterrows():
    to_keep=row[1][1:].values>1
    if any(row[1][1:].values>100):
        fig.add_trace(go.Scatter(x=np.array(range(-1,to_keep.sum())), y=pd.Series(np.append([0],row[1][1:].values[to_keep])).rolling(7).mean().round(),
                        mode='lines',
                        name=row[1][0]))
    #else :
        #print(row[1][0])
fig.update_layout(title='Total number of deaths per country',
                   xaxis_title='Days after the first reported death',
                   yaxis_title='Number of deaths')
fig.show();


# daily death curves

##  Fitted exponential curves

In [191]:
def function(x,a,b):
    return a*np.exp(b*x)

colorscale=['rgb(31, 119, 180)' ,'rgb(255, 127, 14)','rgb(44, 160, 44)','rgb(214, 39, 40)','rgb(148, 103, 189)','rgb(140, 86, 75)']

c=0
fig = go.Figure()
for row in data_format.iterrows():
    to_keep=row[1][1:].values>1
    if (any(row[1][1:].values>100)) & (row[1][0]!='China'): # china has to be removed as it obviously does not fit anymore
        death=pd.Series(np.diff(np.append([0],row[1][1:].values[to_keep]))).rolling(7,center=True).mean().round()
        time=np.array(list(death.index))
        fig.add_trace(go.Scatter(x=time, y=death,line=dict(color=colorscale[c]),name=row[1][0]))
        if len(death.dropna())>5:
            popt, pcov = curve_fit(function,  time[death.notnull()],  death.dropna(),p0=(1,1))
            fig.add_trace(go.Scatter(x=np.linspace(5,time[-1]+5,time[-1]+1), y=pd.Series(function(np.linspace(5,time[-1]+5,time[-1]+1), *popt)).round(),line=dict(color=colorscale[c],dash='dash'),name="Fit - "+row[1][0]))
    #else :
        #print(row[1][0])
     
    c+=1
    if c>(len(colorscale)-1):
        c=0
        
fig.update_layout(title='Daily number of deaths per country',
                   xaxis_title='Days after the first reported death',
                   yaxis_title='Number of deaths')
fig.show();

When the exponential fits badly it means that the propagation is getting mitigated (e.g. Italy,south korea). On the other hand, a really good fit (e.g. Netherlands) means it still propagate exponentially.

## Fitted polynomial curves with 5 days projection

In [187]:
def function(x,*coeffs):
    return  np.polyval(coeffs, x)

colorscale=['rgb(31, 119, 180)' ,'rgb(255, 127, 14)','rgb(44, 160, 44)','rgb(214, 39, 40)','rgb(148, 103, 189)','rgb(140, 86, 75)']

fig = go.Figure()
c=0
for row in data_format.iterrows():
    to_keep=row[1][1:].values>1
    if (any(row[1][1:].values>100)):
        death=pd.Series(np.diff(np.append([0],row[1][1:].values[to_keep]))).rolling(7,center=True).mean().round()
        time=np.array(list(death.index))
        fig.add_trace(go.Scatter(x=time, y=death,line=dict(color=colorscale[c]),name=row[1][0]))
        if len(death.dropna())>5:
            popt, pcov = curve_fit(function,  time[death.notnull()],  death.dropna(),p0=(1,1,1,1,1))
            fig.add_trace(go.Scatter(x=np.linspace(5,time[-1]+5,time[-1]+1), y=pd.Series(function(np.linspace(5,time[-1]+5,time[-1]+1), *popt)).round(),line=dict(color=colorscale[c],dash='dash'),name="Fit - "+row[1][0]))
    #else :
        #print(row[1][0])
    c+=1
    if c>(len(colorscale)-1):
        c=0
fig.update_layout(title='Daily number of deaths per country',
                   xaxis_title='Days after the first reported death',
                   yaxis_title='Number of deaths')
fig.show();

Here is a a way to identify which countries have manage to mitigate the propatation. Where the fitted polynomial curve is droping, we can identiy mitigation. *DISCLAIMER: the projection displayed here are not accurate, they just illustrate the direction of the propagation (up/down) and the pace (angle)* 

5